In [99]:
# Importing important libraries and modules

import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.naive_bayes import GaussianNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer

In [73]:
# base_url tends to be fixed if there are links in the webpage
base_url = 'https://www.mygov.in/'

# next_page part of the url keeps on changing
next_page = base_url + '/group-issue/share-your-ideas-pm-narendra-modis-mann-ki-baat-26th-march-2017/'

# lists that will store the requisite data
suggestions = []
username = []
date = []

# Maximum number of suggestions we need
max_suggestions = 1000

while next_page and len(suggestions) < max_suggestions : 
    
    # Downloading the webpage
    page = requests.get(next_page)
    soup = BeautifulSoup(page.content, 'html.parser')
    #print(soup.prettify())
    
    # Getting the content of the requisite tags
    s = [elem.get_text() for elem in soup.select('div.comment_body')]
    d = [elem.get_text() for elem in soup.select('span.date_time')]
    u = [elem.get_text() for elem in soup.select('span.username')]
    
    # Filling the lists with the requisite values
    for item in s :
        suggestions.append(item)    
    for item in u :
        username.append(item)
    for item in d :
        date.append(item)

    # Checking for the click of 'View More' button on the webpage    
    next_page = soup.find('li', class_='pager-next first last')
    
    # If the button is clicked, the next_page part of the url changes
    if next_page : 
        next_page = base_url + next_page.find('a').get('href')
        
    # Extra - for testing     
    if len(suggestions)%10 == 0:
        print('comments: {}'.format(len(suggestions)))


comments: 10
comments: 20
comments: 30
comments: 40
comments: 50
comments: 60
comments: 70
comments: 80
comments: 90
comments: 100
comments: 110
comments: 120
comments: 130
comments: 140
comments: 150
comments: 160
comments: 170
comments: 180
comments: 190
comments: 200
comments: 210
comments: 220
comments: 230
comments: 240
comments: 250
comments: 260
comments: 270
comments: 280
comments: 290
comments: 300
comments: 310
comments: 320
comments: 330
comments: 340
comments: 350
comments: 360
comments: 370
comments: 380
comments: 390
comments: 400
comments: 410
comments: 420
comments: 430
comments: 440
comments: 450
comments: 460
comments: 470
comments: 480
comments: 490
comments: 500
comments: 510
comments: 520
comments: 530
comments: 540
comments: 550
comments: 560
comments: 570
comments: 580
comments: 590
comments: 600
comments: 610
comments: 620
comments: 630
comments: 640
comments: 650
comments: 660
comments: 670
comments: 680
comments: 690
comments: 700
comments: 710
comments: 720
c

In [74]:
# Creating a Dataframe to store the above extracted list values

column_headers = ["Username","Suggestion","Made On","Category"]
df = pd.DataFrame(columns=column_headers)

# Converting the lists into Series objects **Dataframes store Series objects only**

u = pd.Series(username)
d = pd.Series(date)
s = pd.Series(suggestions)

# Inserting the converted lists as columns in the Dataframe

df["Username"] = u.values
df["Suggestion"] = s.values
df["Made On"] = d.values

df

,Username,Suggestion,Made On,Category
0,Hari Mohan Gupta,Sir my humble submission is that please ask pu...,1 month 3 days ago,NaN
1,paras shah,Hello\n Sir.... Mera AK idea hai Jese bus tic...,1 month 3 days ago,NaN
2,Anu Verma,"Respect sir,\nI am Hindi teacher in one of the...",1 month 3 days ago,NaN
3,Jayesh Kulkarni,Sir\nsuggestions AADHAR BASE SYSTEM\n1.Cash Le...,1 month 3 days ago,NaN
4,AKASH GAUTAM,SIR KINDLY LOOK INTO MARITIME SECTOR SPECIALLY...,1 month 3 days ago,NaN
5,Mamata Singh,महोदय आपके सुशासन में सम्बन्धित मंत्रालय के अध...,1 month 3 days ago,NaN
6,Mamata Singh,महोदय यहां सोचने का विषय यह है कि 9 माह के अन्...,1 month 3 days ago,NaN
7,Mamata Singh,महोदय 9 माह का समय बीत गया न जाने इस अवधि में ...,1 month 3 days ago,NaN
8,Mamata Singh,महोदय 9 माह का समय बीत जाने के बाद भी आज तक C...,1 month 3 days ago,NaN
9,Mamata Singh,"आदरणी प्रधानमंत्री जी,\nभारत सरकार\nमहोदय, साद...",1 month 3 days ago,NaN


In [91]:
# Creating dictionary for Category - Keyword Mapping
dict_category = { 'Medical/Safety/Health/Cleanliness' : {"doctor","nurse","medicine","operation","surgery","accident","ambulance","patient"},
                  'Education/Achievement' : {"teacher","book","education","student","school","admission","english","hindi","subject","college","study","maths","science","engineering","bachelors","mba","medal"},
                  'Agriculture/Farming/Energy' : {"electricity","power","solar","thermal","hydro","farm","crop","land"},
                  'Transport' : {"train","railway","car","bus","airplane","plane","traffic","light","road"},  
                  'Employment': {"job","startup","entrepeneur","salary","stipend","employment"},
                  'Govt Initiatives' : {'aadhar','demonitisation'},
                  'Development/Technology': {"machine","youth","technology","digital","research","robot","import","export"},
                  'Finance/Banking': {"tax","money","gst","cheque","profit","loss","bounce","policy","loan"},
                  'Law/Bribery/Crime': {"law","murder","corruption","jail","rape","stab","police","crime"},
                  'Defence/Security' : {"maritime","marine","navy","cadet","army","soldier","border"}}

# List that stores all the categories
category = []

In [92]:
# Function to find the category based on the words in the comment
def find_category(tok_words):
    
    # Instance to stem the words
    ps=PorterStemmer();
    flag = 0
    for item in dict_category.items(): # item = Categories
        for value in item[1]: # value = keywords
            if ps.stem(value) in tok_words:  
                flag = 1
            if flag == 1:
                return (item[0])
    if flag==0:
        return("Others")   # In case there is no match with any of the keywords   
                

In [77]:
# Function to clean the suggestions
def clean_suggestion( comment ):

    letters_only = re.sub("[^a-zA-Z]", " ", comment) 
    word_list = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))         
    
    ps=PorterStemmer();
    meaningful_words = [ps.stem(w) for w in word_list if not w in stops]   
    
    # Finding the category of the suggestion
    c = find_category(meaningful_words)
    category.append(c)

    # Joining the tokenized words back into one string
    return( " ".join( meaningful_words ))   

In [93]:
# Cleaning all the suggestions, count = 1000
count = df["Suggestion"].size 

# List to store the cleaned data
clean_data = []

for i in xrange( 0, count ):
    clean_data.append( clean_suggestion(df["Suggestion"][i] ) )
    

In [94]:
# Adding a column values of Category in the dataframe
df["Category"] = pd.Series(category)

In [98]:
len(category)
df

,Username,Suggestion,Made On,Category
0,Hari Mohan Gupta,Sir my humble submission is that please ask pu...,1 month 3 days ago,Medical/Safety/Health/Cleanliness
1,paras shah,Hello\n Sir.... Mera AK idea hai Jese bus tic...,1 month 3 days ago,Law/Bribery/Crime
2,Anu Verma,"Respect sir,\nI am Hindi teacher in one of the...",1 month 3 days ago,Education/Achievement
3,Jayesh Kulkarni,Sir\nsuggestions AADHAR BASE SYSTEM\n1.Cash Le...,1 month 3 days ago,Education/Achievement
4,AKASH GAUTAM,SIR KINDLY LOOK INTO MARITIME SECTOR SPECIALLY...,1 month 3 days ago,Defence/Security
5,Mamata Singh,महोदय आपके सुशासन में सम्बन्धित मंत्रालय के अध...,1 month 3 days ago,Others
6,Mamata Singh,महोदय यहां सोचने का विषय यह है कि 9 माह के अन्...,1 month 3 days ago,Others
7,Mamata Singh,महोदय 9 माह का समय बीत गया न जाने इस अवधि में ...,1 month 3 days ago,Others
8,Mamata Singh,महोदय 9 माह का समय बीत जाने के बाद भी आज तक C...,1 month 3 days ago,Others
9,Mamata Singh,"आदरणी प्रधानमंत्री जी,\nभारत सरकार\nमहोदय, साद...",1 month 3 days ago,Others


In [95]:
# TRAINING DATA - first 750 suggestions

train_data = clean_data[:750]

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

# Store the suggestion feature set as an array
# Arrays are easier to deal with
train_data_features = vectorizer.fit_transform(train_data)
train_data_features = train_data_features.toarray()

# Store the categories as an array
train_category = np.array(df["Category"][:750])

distinct_train_category = set(train_category)
print train_data_features.shape
# The output of the above signifies that we have 700 rows of training data
# and each row is associated with 1000 distinct words that are extracted from the suggestions
# These 1000 entries are a bunch of 1s and 0s siginifying the presence and absence of those words
# in that suggestion

(750, 1000)


In [96]:
vocab = vectorizer.get_feature_names()

In [97]:
# TESTING DATA - 750 to 1000 suggestions
test_data = clean_data[750:]

# Same thing being done as above    
test_data_features = vectorizer.transform(test_data)
test_data_features = test_data_features.toarray()

test_category = np.array(df["Category"][750:])

In [84]:
# CLASSIFIERS 

# The results are bound to improve with more data
# Applying Naive Bayes
NB_classifier = GaussianNB()
NB_classifier.fit(train_data_features, train_category)
# Making predictions
expected_category = test_category
predicted_category = NB_classifier.predict(test_data_features)

print(metrics.classification_report(expected_category, predicted_category))
print(metrics.confusion_matrix(expected_category, predicted_category))

             precision    recall  f1-score   support

Agriculture/Farming/Energy                 0.00      0.00      0.00         3
Defence/Security                           0.00      0.00      0.00         2
Development/Technology                     0.26      0.43      0.33        23
Education/Achievement                      0.33      0.47      0.38        32
 Employment                                0.20      0.07      0.10        15
Finance/Banking                            0.26      0.35      0.30        20
Govt Initiatives                           0.00      0.00      0.00         5
Law/Bribery/Crime                          0.25      0.50      0.33        20
Medical/Safety/Health/Cleanliness          0.29      0.29      0.29        14
     Others                                0.69      0.33      0.44       101
  Transport                                0.18      0.13      0.15        15

avg / total                                0.42      0.33      0.34       250

[[ 0  0 

In [86]:
# Applying Random Forest Classifier

RF_classifier = RandomForestClassifier(n_estimators = 100) 
RF_classifier.fit( train_data_features, train_category )
predicted_category = RF_classifier.predict(test_data_features)

print(metrics.classification_report(expected_category, predicted_category))
print(metrics.confusion_matrix(expected_category, predicted_category))

             precision    recall  f1-score   support

Agriculture/Farming/Energy              0.00      0.00      0.00         3
Defence/Security                        0.00      0.00      0.00         2
Development/Technology                  0.88      0.30      0.45        23
Education/Achievement                   0.81      0.91      0.85        32
 Employment                             0.83      0.67      0.74        15
Finance/Banking                         0.78      0.90      0.84        20
Govt Initiatives                        0.00      0.00      0.00         5
Law/Bribery/Crime                       1.00      0.80      0.89        20
Medical/Safety/Health/Cleanliness       0.88      0.50      0.64        14
     Others                             0.74      1.00      0.85       101
  Transport                             0.67      0.40      0.50        15

avg / total                             0.76      0.78      0.74       250

[[  0   0   0   0   0   0   0   0   0   3   

In [87]:
# Applying KNN 

KNN_classifier = KNeighborsClassifier()
KNN_classifier.fit( train_data_features, train_category )
predicted_category = KNN_classifier.predict(test_data_features)

print(metrics.classification_report(expected_category, predicted_category))
print(metrics.confusion_matrix(expected_category, predicted_category))

             precision    recall  f1-score   support

Agriculture/Farming/Energy              0.00      0.00      0.00         3
Defence/Security                        0.00      0.00      0.00         2
Development/Technology                  0.56      0.22      0.31        23
Education/Achievement                   0.86      0.19      0.31        32
 Employment                             1.00      0.07      0.12        15
Finance/Banking                         0.00      0.00      0.00        20
Govt Initiatives                        0.00      0.00      0.00         5
Law/Bribery/Crime                       1.00      0.05      0.10        20
Medical/Safety/Health/Cleanliness       0.00      0.00      0.00        14
     Others                             0.44      0.96      0.60       101
  Transport                             0.17      0.07      0.10        15

avg / total                             0.49      0.44      0.33       250

[[ 0  0  0  0  0  0  0  0  0  3  0]
 [ 0  0 

In [90]:
# CLUSTERING

kmeans = KMeans(n_clusters=10, random_state=0).fit(train_data_features)
print kmeans.labels_
kmeans.predict(test_data_features)

[6 9 6 1 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 8 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 8 6 6 6 6 6 6 6 6 6 6 6 6 6 6 9 6 6 6 6 6 6 6 6 6 6 9 6 6 8 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 8 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 8 6 6 6 8 6 6 6 6 6 6 6 5 6 6 6 5 6 6 5 6 6 6 6 5 6 6 6 6 6 5 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 5 5 6 6 8 6 9 6 6 6 5 6 6 6 5 6 6 6 6 6 6 6 6 5 6
 6 6 6 6 6 6 6 6 6 6 3 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 8 6 6 6 6 6 6 6 9 6 6 6 6 6 6 8 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 3 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 3 6 6 6 6 6 6 8 6 6 6 6 6 3 6 6 6 6 6 6 6 6 6 6
 4 4 8 4 6 6 6 6 6 8 6 4 6 6 6 6 6 6 6 3 6 6 6 6 6 6 9 9 6 6 6 6 6 8 6 6 9
 6 9 6 6 9 6 6 6 6 8 6 6 6 6 3 6 6 6 6 6 6 6 6 6 6 6 6 8 6 6 8 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 9 8 6 6 9 6 6 6 6 6 8 6 6 6 5 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 

/Users/pkaur/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6,
       6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       5, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6,
       8, 6, 6, 8, 9, 8, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 8, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 3, 6,
       6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 9, 6, 8], dtype=int32)